<a href="https://colab.research.google.com/github/jakub1090cn/Analiza-Eksploracyjna-Danych-EDA-/blob/main/Final_Project_supermarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px

**Analiza danych 'Supermarket Sales'**

Analiza została przeprowadzona na podstawie danych dostępnych na: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/data

Zbiór zawiera dane dotyczące sprzedaży w trzech marketach:
- **Invoice id**: Komputerowo generowany numer identyfikacyjny paragonu sprzedażowego.
- **Branch**: Oddział supercentrum (dostępne są 3 oddziały oznaczone jako A, B i C).
- **City**: Lokalizacja supercentrów.
- **Customer type**: Typ klientów, rejestrowany jako Członkowie dla klientów używających karty członkowskiej i Zwykli dla tych bez karty.
- **Gender**: Płeć klienta.
- **Product line**: Ogólne kategorie grupowania produktów - Akcesoria elektroniczne, Akcesoria modowe, Jedzenie i napoje, Zdrowie i uroda, Dom i styl życia, Sport i podróże.
- **Unit price**: Cena każdego produktu w dolarach ($).
- **Quantity**: Liczba produktów zakupionych przez klienta.
- **Tax**: 5% opłata podatkowa od zakupów klienta.
- **Total**: Całkowita cena łącznie z podatkiem.
- **Date**: Data zakupu (dane dostępne od stycznia 2019 do marca 2019).
- **Time**: Czas zakupu (od 10:00 do 21:00).
- **Payment**: Metoda płatności używana przez klienta przy zakupie (dostępne 3 metody – Gotówka, Karta kredytowa i E-wallet).
- **COGS**: Koszt sprzedanych towarów.
- **Gross margin percentage**: Procent marży brutto.
- **Gross income**: Dochód brutto.
- **Rating**: Ocena stratyfikacji klienta dotycząca ich ogólnego doświadczenia z zakupów (w skali od 1 do 10).

In [ ]:
df = pd.read_csv('supermarket_sales.csv')
df.head()


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [ ]:
num_of_purchases = df.groupby('City')['Total'].count().reset_index(name='Purchases')
fig = px.bar(num_of_purchases, x='City', y='Purchases',
                 labels={
                     'Purchases': 'Liczba dokonanych zakupów',
                     'City': 'Miasto'
                 },
                 title='Liczba zakupów z podziałem na miasta')
fig.show()

In [ ]:
revenue_by_city = df.groupby('City')['Total'].sum().reset_index(name='Revenues')
coordinates = {
  'City': ['Mandalay', 'Naypyitaw', 'Yangon'],
  'Lat': [20.98, 20.28, 16.79],
  'Lon': [95.77, 96.27, 96.16]
}
coordinates_df = pd.DataFrame(coordinates)
revenue_geo = pd.merge(revenue_by_city, coordinates_df, on='City')

fig = px.scatter_geo(revenue_geo, lat='Lat', lon='Lon', size='Revenues', color='Revenues',
                     hover_name='City', projection='natural earth',
                     labels={'Revenues': 'Przychody'},
                     title='Przychody z podziałem na miasta na wycinku Azji')

fig.update_geos(
  lonaxis_range=[60, 130],
  lataxis_range=[5, 25],
  showcountries=True,
  countrycolor='black'
)
fig.show()

In [ ]:
avg_purchase_by_rating = df.groupby(['City', 'Rating'])['Total'].mean().reset_index()

fig = px.scatter(avg_purchase_by_rating, x='Rating', y='Total', color='City', trendline='lowess',
                 labels={
                     'Rating': 'Rating',
                     'Total': 'Średnia Kwota Zakupu',
                     'City': 'Miasto'
                 },
                 title='Zależność między ratingiem a średnią kwotą zakupów z uwzględnieniem miast')
fig.show()

Jak widać na wykresie powyżej, średni rating nie przekłada się na średnią kwotę zakupów.

In [ ]:
avg_rating_by_city = df.groupby('City')['Rating'].mean().reset_index()
fig = px.bar(avg_rating_by_city, x='City', y='Rating',
                 labels={
                     'Rating': 'Rating',
                     'City': 'Miasto'
                 },
                 title='Średni Rating klientów z podziałem na miasta')
fig.show()

Na wykresie słupkowym można zauwayć, że Naypitaw ma największy średni rating, a najmnijeszy Mandalay.

In [ ]:
fig = px.violin(df, x='City', y='Rating',
                 labels={
                     'Rating': 'Rating',
                     'City': 'Miasto'
                 },
                 title='Rozkład Ratingu klientów z podziałem na miasta')
fig.show()

Natomiat rozkład ratingu z podziałem na miasta wygląda podobnie w każdym z trzech przypadków.

**Test statystyczny ANOVA**

Na wykresach wyraźnie widać, że sklep w Mandalay ma najniższą średnią ocenę. Na tej podstawie można założyć, że oceny sklepu w Mandalay są niższe niż sklepów w pozostałych miastach. Przeprowadzę test statystyczny mający potwierdzić tę obserwację.
Do przeprowadzenia testu ANOVA wymagane jest sprawdzenie warunków:
- zmienne muszą mieć rozkład normalny, co można sprawdzić testem Shapiro-Wilka,
- brak outlierów, który można stwierdzić na podstawie wykresu pudełkowego,
- homogeniczność wariancji między grupami weryfikowana testem Leven'a
- równoliczność grup.

W pierwszej kolejności sprawdzę powyższe założenia:

In [ ]:
import scipy.stats as stats

filtered_df = df[df['City'].isin(['Mandalay', 'Naypyitaw', 'Yangon'])]

print('Statystyka Shapiro-Wilka:')
for city in filtered_df['City'].unique():
    result = stats.shapiro(filtered_df[filtered_df['City'] == city]['Rating'])
    print(f'City: {city},  p-value: {result.pvalue}')

result = stats.levene(filtered_df[filtered_df['City'] == 'Mandalay']['Rating'],
                       filtered_df[filtered_df['City'] == 'Naypyitaw']['Rating'],
                       filtered_df[filtered_df['City'] == 'Yangon']['Rating'])
print(f'\nTest Levene\'a, p-value: {result.pvalue}\n')

fig = px.box(filtered_df, x='City', y='Rating', title='Sprawdzenie outlierów')
fig.show()

print(filtered_df.groupby('City').size())


Statystyka Shapiro-Wilka:
City: Yangon,  p-value: 1.5700766908821606e-08
City: Naypyitaw,  p-value: 2.8268669183262318e-08
City: Mandalay,  p-value: 4.01483220002774e-08

Test Levene'a, p-value: 0.9583492264513054



City
Mandalay     332
Naypyitaw    328
Yangon       340
dtype: int64


Założenia testu ANOVA nie są spełnione. Warunek równoliczności grup nie pozwala nam wykorzystać tego testu. Z tego powodu wykonam test Kruskala-Wallisa:

In [ ]:
result = stats.kruskal(filtered_df[filtered_df['City'] == 'Mandalay']['Rating'],
                       filtered_df[filtered_df['City'] == 'Naypyitaw']['Rating'],
                       filtered_df[filtered_df['City'] == 'Yangon']['Rating'])

print('Statystyka testu Kruskala-Wallisa:', result.statistic)
print('P-wartość:', result.pvalue)

Statystyka testu Kruskala-Wallisa: 4.086085882493156
P-wartość: 0.12963364254056914


'p-value' jest większe od 0,05. Nie ma podstaw do stwierdzenia, że między ratingiem poszczególnych sklepów istnieje różnica istotna statystycznie.

In [ ]:
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M')
df['Hour'] = df['Time'].dt.hour

count_gender_hour = df.groupby(['Gender', 'Hour']).size().reset_index(name='Liczba osób')


fig = px.line(count_gender_hour, x='Hour', y='Liczba osób', color='Gender',
              labels={'Hour': 'Godzina', 'Gender': 'Płeć', 'Liczba osób': 'Liczba osób robiących zakupy'},
              title='Liczba dokonanych zakupów w poszczególnych godzinach z podziałem na płeć')

fig.update_traces(mode='lines+markers')
fig.show()

Wykres liniowy wskazuje, że do południa więcej zakupów dokonują kobiety, natomiast po południu dominują mężczyźni. Warto zwrócić uwagę na fakt, że piki zakupowe mają miejsce cztery razy w ciągu dnia.

In [ ]:
count_income = df.groupby('Hour')['Total'].count().reset_index(name='Revenue')
fig = px.line(count_income, x='Hour', y='Revenue',
             labels={'Hour': 'Godzina', 'Income': 'Przychód'},
             title='Przychód w poszczególnych godzinach')
fig.show()

Czas największych przychodów przypada na godziny, w których najwięcej dokonywanych jest najwięcej transakcji.

In [ ]:
count_gender_city = df.groupby(['City', 'Gender']).size().reset_index(name='Liczba osób')
fig = px.bar(count_gender_city, x='City', y='Liczba osób', color='Gender', barmode='group',
             labels={'City': 'Miasto', 'Gender': 'Płeć', 'Liczba osób': 'Liczba osób robiących zakupy'},
             title='Liczba osób robiących zakupy w poszczególnych miastach z podziałem na płeć')
fig.show()

Liczby mężczyzn i kobiet robiących zakupy w poszczególnych sklepach są zbliżone do siebie, jednak można zauważyć, że w Naypyitaw występuje przewaga kobiet, a w pozostałych miastach nieduża przewaga mężczyzn

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
products_by_gender = df.groupby(['Gender', 'Product line']).size().reset_index(name='Liczba zakupów')

fig = px.bar(products_by_gender, x='Product line', y='Liczba zakupów', color='Gender', barmode='group',
             labels={'Product line': 'Produkty', 'Liczba zakupów': 'Liczba zakupów', 'Gender': 'Płeć'},
             title='Liczba zakupionych artykułów z podziałem na płeć')

fig.show()

Powyższy wykres wskazuje, że mężczyźni zdecydowanie częściej kupują produkty związane ze zdrowiem i urodą. Kobiety najczęściej kupują akcesoria modowe. W pozostałych przypadkach liczba zakupów kobiet i mężczyzn jest porównywalna.

In [ ]:
count_member_city = df.groupby(['City', 'Customer type']).size().reset_index(name='Liczba osób')
fig = px.bar(count_member_city, x='City', y='Liczba osób', color='Customer type', barmode='group',
             labels={'City': 'Miasto', 'Customer type': 'Typ Klienta', 'Liczba osób': 'Liczba osób'},
             title='Liczba osób posiadających kartę członkowską i nie, w poszczególnych miastach')
fig.show()

Zakupy w badanych sklepach są dokonywane przez klientów zwykłych i członków w podobnej ilości. Jedynie w Naypyitaw występuje niewielka przewaga członków nad klientami zwykłymi.

In [ ]:
count_products = df.groupby(['Customer type', 'Product line']).size().reset_index(name='Liczba zakupów')
fig = px.bar(count_products, x='Product line', y='Liczba zakupów', color='Customer type', barmode='group',
             title='Porównanie zakupów produktów przez członków i zwykłych klientów')
fig.show()

Jak widać na wykresie, klienci mający status członka częściej kupują artykuły spożywcze, sportowe i związane z domem i style życia. W pozostałych przypadkach posiadanie statusu członka nie wpływa na zwiększoną ilość zakupów.

In [ ]:
fig = px.box(df, x='Product line', y='cogs',
             labels={'Product line': 'Rodzaj artykułów', 'cogs': 'Cena zakupów'},
             title='Cena pojedynczych zakupów a rodzaj artykułów')
fig.show()

Pojeyncze zakupy związane z poszczególnymi kategoriami produktów są zbliżone cenowo. Zwracając uwagę na górną granicę można wywnioskować, że częściej zdaża się klientom zapłacić więcej za produkty związane z domem i stylem życia. Natomiast mediana tych zakupów nie wyróżnia się spośród pozostałych kategorii produktowych.

In [ ]:
fig = px.box(df, x='Customer type', y='Total',
             labels={'Total': 'Cena zakupów', 'Customer type': 'Rodzaj klienta'},
             title='Cena jednorazowych zakupów a rodzaj klienta')
fig.show()

Biorąc pod uwagę cenę zakupu i status klienta można zaobserwować, że członkowie wydają więcej pieniędzy na jednorazowe zakupy.

In [ ]:
fig = px.box(df, x='Payment', y='cogs',
             labels={'Payment': 'Rodzaj płatności', 'cogs': 'Wartość rachunku'},
             title='Cena jednorazowych zakupów a rodzaj płatności')
fig.show()

Rodzaj płatności nie ma wpływu na ilość pieniędzy zostawionych w sklepie. Co prawda płatności kartą kredytową mają wyższe maksymalne wartości, natomiast mediana wskazuje, że pozostałe rodzaje płatności wypadają lepiej.

In [ ]:
payment_count = df.groupby('Payment').size().reset_index(name='sum_of_payments')

fig = px.bar(payment_count, x='Payment', y='sum_of_payments',
             labels={'sum_of_payments': 'Liczba płatności', 'Payment': 'Metoda płatności'},
             title='Popularność poszczególnych metod płatności')
fig.show()

Płatności kartą kredytową są najmniej popularne. Ewallet i płatności gotówkowe są niemal na tym samym poziomie.

In [ ]:
revenues_payment = df.groupby('Payment')['Total'].sum().reset_index(name='revenues by payment method')
fig = px.bar(revenues_payment, x='Payment', y= 'revenues by payment method',
             labels={'Payment': 'Przygody', 'revenues by payment method': 'Metoda płatności'},
             title='Suma przychodów z podziałem na metody płatności')
fig.show()

Najwięcej przychodów generują płatności gotówkowe.

Popularnym w mediach zjawiskiem jest efekt stycznia, kiedy w wyniku postanowień noworocznych można zaobserwować wśród ludzi zwiększoną aktywność fizyczną, która następnie wraca do normalnego poziomu. Chcę sprawdzić, czy te zjawisko przekłada się na zakupy związane z artykułami sportowymi.

In [ ]:
filtered = df[df['Product line'] == 'Sports and travel'].copy()
filtered['Week'] = filtered['Date'].dt.isocalendar().week.astype(int)
grouped_df = filtered.groupby('Week')['Total'].sum().reset_index(name='Weekly Revenues')
fig = px.scatter(grouped_df, x='Week', y='Weekly Revenues', trendline='lowess',
                 labels={'Weekly Revenues': 'Przychody tygodniowe', 'Week': 'Tydzień'},
                 title='Łączny przychód z "Sports and travel" każdego tygodnia od początku roku')

fig.show()

Jak widać przychód generowany ze sprzedaży artykułów sportowych na początku roku jest zbliżony do przychodu z końca marca. Natomiast można zauważyć zwiąkszony wpływ w czwartym tygodniu stycznia, co wskazywałoby, że faktycznie postanowienia noworoczne przekładają się na ilość wydawanych pieniędzy.

In [ ]:
weekly_counts = filtered.groupby('Week').size().reset_index(name='Liczba Zakupów')
fig = px.scatter(weekly_counts, x='Week', y='Liczba Zakupów', trendline='lowess',
                 labels={'Week':'Tydzień'},
                 title='Liczba zakupów "Sports and travel" każdego tygodnia')
fig.show()


W przypadku liczby zakupów widzimy wyraźną tendencję spadkową zakupów związanych ze sportem i podróżami zaczynającą się z końcem stycznia i trwającą do końca marca.